# Teste KPIs

## Conexão

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window

## Leitura dos Dados

In [0]:
df_exp = spark.sql("select * from hive_metastore.silver.exp")
df_imp = spark.sql("select * from hive_metastore.silver.imp")
df_ncm = spark.sql("select * from hive_metastore.silver.ncm")
df_ncm_fat_agreg = spark.sql("select * from hive_metastore.silver.ncm_fat_agreg")

In [0]:
#FATOS
# Exportações por município (maior granularidade)
df_exp_mun = spark.sql("SELECT * FROM hive_metastore.silver.exp_mun")
df_imp_mun = spark.sql("SELECT * FROM hive_metastore.silver.imp_mun")
# Dimensão País
df_pais = spark.sql("SELECT * FROM hive_metastore.silver.pais").select("CO_PAIS","NO_PAIS")
# Dimensão País x Bloco Econômico
df_pais_bloco = spark.sql("SELECT * FROM hive_metastore.silver.pais_bloco").select("CO_PAIS","NO_BLOCO")
# Dimensão Via de Transporte
df_via = spark.sql("SELECT * FROM hive_metastore.silver.via").select("CO_VIA","NO_VIA")
# Dimensão Unidade Federativa
df_uf = spark.sql("SELECT * FROM hive_metastore.silver.uf")

In [0]:
PATH_GOLD = f"abfss://gold@aceleragrupo5sa.dfs.core.windows.net/"

In [0]:
def salva_tabela(df,nome_tabela,PATH):

    (
        df.write
            .format("delta")
            .mode("overwrite")
            .save(PATH + nome_tabela)
    )

    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS gold.{nome_tabela}
        USING DELTA
        LOCATION '{PATH}{nome_tabela}'
    """)

## Tabelas Auxiliares

### NCM

In [0]:
df_dim_ncm = df_ncm.select("CO_NCM","NO_NCM_POR").distinct()
df_dim_ncm.display()

In [0]:
salva_tabela(df_dim_ncm,"dim_ncm",PATH_GOLD)

### Data

In [0]:
df_dim_data = df_exp.select("ANO", "MES").distinct()
df_dim_data = df_dim_data.withColumn("DATA", 
                F.to_date(F.concat_ws("-", F.col("ANO"), F.lpad(F.col("MES"), 2, "0"), F.lit("01")), "yyyy-MM-dd"))

df_dim_data.display()

In [0]:
salva_tabela(df_dim_data,"dim_data",PATH_GOLD)

## KPIs - Painel 1

### 1. Volume Total de Exportações e Importações (USD FOB)
### 2. Evolução do Saldo Comercial (Superávit / Déficit) por mês



In [0]:
# Trazendo os dados

df_vl_exp = df_exp.select("ANO", "MES", F.col("VL_FOB").alias("VL_FOB_EXP"))
df_vl_imp = df_imp.select("ANO", "MES",  F.col("VL_FOB").alias("VL_FOB_IMP"))

# Agregação por ano e mês

df_vl_exp = df_vl_exp.groupBy(["ANO", "MES"]).agg(F.sum("VL_FOB_EXP").alias("VL_FOB_EXP"))
df_vl_imp = df_vl_imp.groupBy(["ANO", "MES"]).agg(F.sum("VL_FOB_IMP").alias("VL_FOB_IMP"))

# Join

df_vl_exp_imp = df_vl_exp.join(df_vl_imp, ["ANO", "MES"], "full_outer")

# Adicionando Saldo

df_saldo = df_vl_exp_imp.withColumn("VL_SALDO_COMERCIAL", F.col("VL_FOB_EXP") - F.col("VL_FOB_IMP"))

# Coluna de Data

df_saldo = df_saldo.withColumn("DATA", 
            F.to_date(F.concat_ws("-", F.col("ANO"), F.lpad(F.col("MES"), 2, "0"), F.lit("01")), "yyyy-MM-dd"))

# view

view_vol_exp_imp_saldo = df_saldo.select("DATA", "VL_FOB_EXP", "VL_FOB_IMP", "VL_SALDO_COMERCIAL")

In [0]:
view_vol_exp_imp_saldo.display()

In [0]:
salva_tabela(view_vol_exp_imp_saldo,"view_painel1_vol_exp_imp_saldo",PATH_GOLD)

### 3. Participação Relativa: Exportação vs Importação (%) por Ano

In [0]:
view_pct_exp_imp = df_vl_exp_imp.groupBy("ANO") \
    .agg(F.sum("VL_FOB_EXP").alias("VL_FOB_EXP"), 
         F.sum("VL_FOB_IMP").alias("VL_FOB_IMP"))


In [0]:
view_pct_exp_imp.display()

In [0]:
salva_tabela(view_pct_exp_imp,"view_painel1_pct_exp_imp",PATH_GOLD)

### 4. Distribuição por Grau de Industrialização (Fator Agregado)

In [0]:
# Trazendo os dados

df_vl_exp_ncm = df_exp.select("ANO","CO_NCM", F.col("VL_FOB").alias("VL_FOB_EXP"))

df_vl_imp_ncm = df_imp.select("ANO","CO_NCM", F.col("VL_FOB").alias("VL_FOB_IMP"))

df_vl_ncm = df_ncm.select("CO_NCM", "CO_FAT_AGREG")
df_fat_agreg = df_ncm_fat_agreg.select("CO_FAT_AGREG", "NO_FAT_AGREG")

df_vl_ncm_fat_agregado = df_vl_ncm.join(df_fat_agreg, "CO_FAT_AGREG", "inner") \
                                  .select("CO_NCM","NO_FAT_AGREG")

In [0]:
# Agrupar por produto e somar valores

df_vl_exp_ncm = df_vl_exp_ncm.groupBy("ANO","CO_NCM").agg(F.sum("VL_FOB_EXP").alias("VL_FOB_EXP"))
df_vl_imp_ncm = df_vl_imp_ncm.groupBy("ANO","CO_NCM").agg(F.sum("VL_FOB_IMP").alias("VL_FOB_IMP"))

# Join para trazer o fator agregado

df_vl_exp_ncm = df_vl_exp_ncm.join(df_vl_ncm_fat_agregado, "CO_NCM", "inner").drop("CO_NCM")
df_vl_imp_ncm = df_vl_imp_ncm.join(df_vl_ncm_fat_agregado, "CO_NCM", "inner").drop("CO_NCM")

# Agrupar por fator agregado

df_vl_exp_ncm = df_vl_exp_ncm.groupBy("ANO","NO_FAT_AGREG").agg(F.sum("VL_FOB_EXP").alias("VL_FOB_EXP"))
df_vl_imp_ncm = df_vl_imp_ncm.groupBy("ANO","NO_FAT_AGREG").agg(F.sum("VL_FOB_IMP").alias("VL_FOB_IMP"))

In [0]:
# Trazer porcentagem por fator de agregação

window = Window.rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

df_vl_fat_perc_exp = (
    df_vl_exp_ncm
    .withColumn("TOTAL_GERAL", F.sum("VL_FOB_EXP").over(window))
    .withColumn(
        "PERC_TOTAL_EXP",
        F.round((F.col("VL_FOB_EXP") / F.col("TOTAL_GERAL") * 100),2)
    )
).drop("TOTAL_GERAL")

df_vl_fat_perc_imp = (
    df_vl_imp_ncm
    .withColumn("TOTAL_GERAL", F.sum("VL_FOB_IMP").over(window))
    .withColumn(
        "PERC_TOTAL_IMP",
        F.round((F.col("VL_FOB_IMP") / F.col("TOTAL_GERAL") * 100),2)
    )
).drop("TOTAL_GERAL")

# Join

df_vl_fat_perc_exp_imp = df_vl_fat_perc_exp.join(df_vl_fat_perc_imp,["ANO","NO_FAT_AGREG"], "inner")
df_vl_fat_perc_exp_imp = df_vl_fat_perc_exp_imp.withColumn(
    "NO_FAT_AGREG",
    F.trim(F.regexp_replace(F.col("NO_FAT_AGREG"), "PRODUTOS", ""))
)

# view

view_pct_fat_agreg = df_vl_fat_perc_exp_imp \
    .select("ANO", "NO_FAT_AGREG", "VL_FOB_EXP", "PERC_TOTAL_EXP", "VL_FOB_IMP", "PERC_TOTAL_IMP")

In [0]:
view_pct_fat_agreg.display()

In [0]:
salva_tabela(view_pct_fat_agreg,"view_painel1_pct_fat_agreg",PATH_GOLD)

## KPIs - Painel 2

### 1. NCMs mais Exportados e Importados (Valor FOB)
### 2. Volume x Valor agregado (Preço Médio) por produto

In [0]:
# Trazendo os dados

df_vl_exp_ncm = df_exp.select("CO_NCM", F.col("VL_FOB").alias("VL_FOB_EXP"), "KG_LIQUIDO")
df_vl_imp_ncm = df_imp.select("CO_NCM", F.col("VL_FOB").alias("VL_FOB_IMP"), "KG_LIQUIDO")

In [0]:
# Agregação por produto trazendo o preço médio

df_vl_exp_ncm = df_vl_exp_ncm.groupBy("CO_NCM") \
                .agg(F.sum("KG_LIQUIDO").alias("KG_LIQUIDO_EXP"),
                     F.sum("VL_FOB_EXP").alias("VL_FOB_EXP")) \
                .withColumn("PRECO_MEDIO_EXP", 
                            F.round(F.col("VL_FOB_EXP") / F.col("KG_LIQUIDO_EXP"),2))
                
df_vl_imp_ncm = df_vl_imp_ncm.groupBy("CO_NCM") \
                .agg(F.sum("KG_LIQUIDO").alias("KG_LIQUIDO_IMP"),
                     F.sum("VL_FOB_IMP").alias("VL_FOB_IMP")) \
                .withColumn("PRECO_MEDIO_IMP", 
                            F.round(F.col("VL_FOB_IMP") / F.col("KG_LIQUIDO_IMP"),2))

# Joins

df_vl_exp_imp_ncm = df_vl_exp_ncm.join(df_vl_imp_ncm, "CO_NCM", "full_outer")

# Preencher nulos

df_vl_exp_imp_ncm = df_vl_exp_imp_ncm \
                .withColumn("VL_FOB_EXP", F.coalesce(F.col("VL_FOB_EXP"), F.lit(0))) \
                .withColumn("VL_FOB_IMP", F.coalesce(F.col("VL_FOB_IMP"), F.lit(0))) \
                .withColumn("KG_LIQUIDO_EXP", F.coalesce(F.col("KG_LIQUIDO_EXP"), F.lit(0))) \
                .withColumn("KG_LIQUIDO_IMP", F.coalesce(F.col("KG_LIQUIDO_IMP"), F.lit(0))) \
                .withColumn("PRECO_MEDIO_EXP", F.coalesce(F.col("PRECO_MEDIO_EXP"), F.lit(0))) \
                .withColumn("PRECO_MEDIO_IMP", F.coalesce(F.col("PRECO_MEDIO_IMP"), F.lit(0))) \
                .select("CO_NCM", "VL_FOB_EXP", "KG_LIQUIDO_EXP", "PRECO_MEDIO_EXP","VL_FOB_IMP", "KG_LIQUIDO_IMP",  "PRECO_MEDIO_IMP")

# view

view_ncm_fob_preco_med = df_vl_exp_imp_ncm.select("CO_NCM", "VL_FOB_EXP", "KG_LIQUIDO_EXP", "PRECO_MEDIO_EXP","VL_FOB_IMP", "KG_LIQUIDO_IMP",  "PRECO_MEDIO_IMP")

In [0]:
view_ncm_fob_preco_med.display()

In [0]:
salva_tabela(view_ncm_fob_preco_med,"view_painel2_ncm_fob_preco_med",PATH_GOLD)

### 3. Variações (%) de produtos mês a mês

In [0]:
# Trazendo os dados

df_vl_exp_ncm = df_exp.select("ANO", "MES", "CO_NCM", F.col("VL_FOB").alias("VL_FOB_EXP"))
df_vl_imp_ncm = df_imp.select("ANO", "MES", "CO_NCM", F.col("VL_FOB").alias("VL_FOB_IMP"))

# Agregação por produto e soma de valores

df_vl_exp_ncm = df_vl_exp_ncm.groupBy("ANO", "MES", "CO_NCM") \
                             .agg(F.sum("VL_FOB_EXP").alias("VL_FOB_EXP"))
                             
df_vl_imp_ncm = df_vl_imp_ncm.groupBy("ANO", "MES", "CO_NCM") \
                             .agg(F.sum("VL_FOB_IMP").alias("VL_FOB_IMP"))

df_vl_exp_imp_ncm = df_vl_exp_ncm.join(df_vl_imp_ncm, ["ANO", "MES", "CO_NCM"], "full_outer")

In [0]:
# Janela por produto, ordenada por ano e mês

window = (Window.partitionBy("CO_NCM").orderBy("ANO", "MES"))

df_vl_exp_imp_ncm = (
    df_vl_exp_imp_ncm
    .withColumn("VL_FOB_EXP_LAG", F.lag("VL_FOB_EXP").over(window))
    .withColumn("VAR_PERC_MES_EXP",
                F.when(
                    F.col("VL_FOB_EXP_LAG").isNull(), None
                ).otherwise(
                    F.round(((F.col("VL_FOB_EXP") - F.col("VL_FOB_EXP_LAG")) 
                    / F.col("VL_FOB_EXP_LAG") * 100),2)
                )) \
    .withColumn("VL_FOB_IMP_LAG", F.lag("VL_FOB_IMP").over(window))
    .withColumn("VAR_PERC_MES_IMP",
                F.when(
                    F.col("VL_FOB_IMP_LAG").isNull(), None
                ).otherwise(
                    F.round(((F.col("VL_FOB_IMP") - F.col("VL_FOB_IMP_LAG")) 
                    / F.col("VL_FOB_IMP_LAG") * 100),2)
                ))
    ).drop("VL_FOB_EXP_LAG")


In [0]:
# Preencher nulos

df_vl_exp_imp_ncm = df_vl_exp_imp_ncm \
                .withColumn("VL_FOB_EXP", F.coalesce(F.col("VL_FOB_EXP"), F.lit(0))) \
                .withColumn("VL_FOB_IMP", F.coalesce(F.col("VL_FOB_IMP"), F.lit(0))) \
                .withColumn("VAR_PERC_MES_EXP", F.coalesce(F.col("VAR_PERC_MES_EXP"), F.lit(0))) \
                .withColumn("VAR_PERC_MES_IMP", F.coalesce(F.col("VAR_PERC_MES_IMP"), F.lit(0))) \
                .select("ANO", "MES", "CO_NCM", "VL_FOB_EXP", "VAR_PERC_MES_EXP","VL_FOB_IMP", "VAR_PERC_MES_IMP")

# Coluna de Data

df_vl_exp_imp_ncm = df_vl_exp_imp_ncm \
    .withColumn("DATA", F.to_date(F.concat_ws("-", F.col("ANO"), F.lpad(F.col("MES"), 2, "0"), F.lit("01")), "yyyy-MM-dd"))

# view

view_ncm_var_mes = df_vl_exp_imp_ncm.select("DATA", "CO_NCM", "VL_FOB_EXP", "VAR_PERC_MES_EXP","VL_FOB_IMP", "VAR_PERC_MES_IMP")

In [0]:
view_ncm_var_mes.orderBy("CO_NCM", "DATA").display()

In [0]:
salva_tabela(view_ncm_var_mes,"view_painel2_ncm_var_mes",PATH_GOLD)

### Filtro 10 produtos mais exportados e importados

In [0]:
df_exp_rank = view_ncm_fob_preco_med.orderBy(F.desc("VL_FOB_EXP")).select("CO_NCM", "VL_FOB_EXP")
df_imp_rank = view_ncm_fob_preco_med.orderBy(F.desc("VL_FOB_IMP")).select("CO_NCM", "VL_FOB_IMP")

top5_exp = (df_exp_rank.select("CO_NCM").limit(5).withColumn("origem", F.lit("EXPORTAÇÃO")))
top5_imp = (df_imp_rank.select("CO_NCM").limit(5).withColumn("origem", F.lit("IMPORTAÇÃO")))

base_top = (top5_exp.union(top5_imp).dropDuplicates(["CO_NCM"]))

# contar quantos temos na base depois de retirar duplicatas

qtd_atual = base_top.count()
faltam = 10 - qtd_atual

# buscar NCMs extras da base de exportação

extras_exp = (
    df_exp_rank
    .orderBy(F.desc("VL_FOB_EXP"))
    .join(base_top.select("CO_NCM"),"CO_NCM","left_anti")
    .limit(faltam)
    .select("CO_NCM")
    .withColumn("origem", F.lit("EXPORTAÇÃO_EXTRA"))
)

# recalcular

base_com_exp = base_top.union(extras_exp)
qtd_atual = base_com_exp.count()
faltam = 10 - qtd_atual

# se ainda faltar buscar da base de importação

extras_imp = (
    df_imp_rank
    .orderBy(F.desc("VL_FOB_IMP"))
    .join(base_com_exp.select("CO_NCM"),"CO_NCM","left_anti")
    .limit(faltam)
    .select("CO_NCM")
    .withColumn("origem", F.lit("IMPORTAÇÃO_EXTRA"))
)

top10_final = (base_com_exp.union(extras_imp).limit(10))

In [0]:
top10_final.display()

## KPIs - Painel 3

In [0]:
#Criando duas bases temporais espelhadas (exportação e importação)
#Isolando somente o que importa para análise de valor ao longo do tempo
#Evitando conflitos de nomes (VL_FOB_EXP vs VL_FOB_IMP)
#Garantindo tipagem correta para cálculos, joins e gráficos

df_vl_exp = df_exp.select(
    "ano",
    "mes",
    F.col("vl_fob").alias("VL_FOB_EXP")
)

df_vl_imp = df_imp.select(
    "ano",
    "mes",
    F.col("vl_fob").alias("VL_FOB_IMP")
)


### Top 10 países em exportação

In [0]:
df_produtos_top10_exp = (
    top10_final
    .select(F.col("co_ncm").alias("produto_ncm"))
    .distinct()
)

df_exp_filtrado = (
    df_exp
    .join(
        df_produtos_top10_exp,
        df_exp.co_ncm == df_produtos_top10_exp.produto_ncm,
        how="inner"
    )
).select("produto_ncm","co_pais","vl_fob")

df_exp_total = (
    df_exp_filtrado
    .groupBy("co_pais", "produto_ncm")
    .agg(
        F.sum("vl_fob").alias("valor_exportado")
    )
)

df_exp_total = (
    df_exp_total
    .join(df_pais, on="co_pais", how="left")
)

df_top10_exportacao_pais = (
    df_exp_total
    .orderBy(F.desc("valor_exportado"))
    .select(
        F.col("no_pais"),
        F.col("produto_ncm"),
        F.col("valor_exportado")
    )
)

In [0]:
df_top10_exportacao_pais.display()

In [0]:
salva_tabela(df_top10_exportacao_pais,"view_painel3_rank_exportacao_pais",PATH_GOLD)

### Top 10 países em importação

In [0]:
df_produtos_top10_imp = (
    top10_final
    .select(F.col("co_ncm").alias("produto_ncm"))
    .distinct()
)

df_imp_filtrado = (
    df_imp
    .join(
        df_produtos_top10_imp,
        df_imp.co_ncm == df_produtos_top10_imp.produto_ncm,
        how="inner"
    )
).select("produto_ncm","co_pais","vl_fob")

df_imp_total = (
    df_imp_filtrado
    .groupBy("co_pais", "produto_ncm")
    .agg(
        F.sum("vl_fob").alias("valor_importado")
    )
)

df_imp_total = (
    df_imp_total
    .join(df_pais, on="co_pais", how="left")
)


df_top10_importacao_pais = (
    df_imp_total
    .orderBy(F.desc("valor_importado"))
    .select(
        F.col("no_pais"),
        F.col("produto_ncm"),
        F.col("valor_importado")
    )
)


In [0]:
df_top10_importacao_pais.display()

In [0]:
salva_tabela(
    df_top10_importacao_pais,
    "view_painel3_rank_importacao_pais",
    PATH_GOLD
)


### Exportação e Importação por Bloco

In [0]:
# EXPORTAÇÃO POR BLOCO ECONÔMICO

df_exp_bloco = (
    df_exp
    .join(df_pais_bloco, on="co_pais", how="left")
    .groupBy("NO_BLOCO","co_ncm")
    .agg(
        F.sum("vl_fob").alias("valor_fob")
    )
    .withColumn("tipo_operacao", F.lit("EXPORTACAO"))
).select("NO_BLOCO","co_ncm","valor_fob","tipo_operacao")

df_exp_bloco = (
    df_exp_bloco
    .join(
        df_produtos_top10_imp,
        df_exp_bloco.co_ncm == df_produtos_top10_imp.produto_ncm,
        how="inner"
    )
).select("NO_BLOCO","co_ncm","valor_fob","tipo_operacao")


# IMPORTAÇÃO POR BLOCO ECONÔMICO

df_imp_bloco = (
    df_imp
    .join(df_pais_bloco, on="co_pais", how="left")
    .groupBy("NO_BLOCO", "co_ncm")
    .agg(
        F.sum("vl_fob").alias("valor_fob")
    )
    .withColumn("tipo_operacao", F.lit("IMPORTACAO"))
).select("NO_BLOCO","co_ncm","valor_fob","tipo_operacao")

df_imp_bloco = (
    df_imp_bloco
    .join(
        df_produtos_top10_imp,
        df_imp_bloco.co_ncm == df_produtos_top10_imp.produto_ncm,
        how="inner"
    )
).select("NO_BLOCO","co_ncm","valor_fob","tipo_operacao")


# UNION EXPORTAÇÃO + IMPORTAÇÃO

df_comercio_bloco = df_exp_bloco.unionByName(df_imp_bloco)

df_comercio_bloco.display()


In [0]:
salva_tabela(df_comercio_bloco,"view_painel3_comercio_bloco",PATH_GOLD)

### Saldo

In [0]:
#saldo exportacao
df_exp_uf = (
    df_exp
    .groupBy("sigla_uf_ncm","co_ncm")
    .agg(
        F.sum(F.col("vl_fob").cast("double")).alias("VL_FOB_EXP")
    )
)

#saldo importacao
df_imp_uf = (
    df_imp
    .groupBy("sigla_uf_ncm","co_ncm")
    .agg(
        F.sum(F.col("vl_fob").cast("double")).alias("VL_FOB_IMP")
    )
)

#saldo total
df_saldo_uf = (
    df_exp_uf
    .join(df_imp_uf, ["sigla_uf_ncm","co_ncm"], "full_outer")
    .withColumn(
        "SALDO_COMERCIAL",
        F.coalesce(F.col("VL_FOB_EXP"), F.lit(0)) -
        F.coalesce(F.col("VL_FOB_IMP"), F.lit(0))
    )
    .join(df_uf, df_uf.sg_uf == F.col("sigla_uf_ncm"), "left")
    .select(
        "sigla_uf_ncm",
        "co_ncm",
        "no_uf",
        "no_regiao",
        "VL_FOB_EXP",
        "VL_FOB_IMP",
        "SALDO_COMERCIAL"
    )
)

df_saldo_uf = (
    df_saldo_uf
    .join(
        df_produtos_top10_imp,
        df_saldo_uf.co_ncm == df_produtos_top10_imp.produto_ncm,
        how="inner"
    )
).drop("produto_ncm")

In [0]:
df_saldo_uf.display()

In [0]:
salva_tabela(df_saldo_uf,"view_painel3_saldo_comercial_uf",PATH_GOLD)

### Volume Exportação e Importação por Via

In [0]:
df_volume_exp = (
    df_exp
    .groupBy("co_via", "co_ncm")
    .agg(
        F.sum(F.col("kg_liquido").cast("double")).alias("kg_exp"),
        F.sum(F.col("vl_fob").cast("double")).alias("vl_fob_exp")
    )
    .join(df_via, "CO_VIA", "left")
)

df_volume_exp = (
    df_volume_exp
    .join(
        df_produtos_top10_imp,
        df_volume_exp.co_ncm == df_produtos_top10_imp.produto_ncm,
        how="inner"
    )
).drop("produto_ncm")



In [0]:
df_volume_exp.display()

In [0]:
salva_tabela(df_volume_exp,"view_painel3_volume_exp",PATH_GOLD)

In [0]:
df_volume_imp = (
    df_imp
    .groupBy("co_via","co_ncm")
    .agg(
        F.sum(F.col("kg_liquido").cast("double")).alias("kg_imp"),
        F.sum(F.col("vl_fob").cast("double")).alias("vl_fob_imp"),
    )
    .join(df_via, "CO_VIA", "left")
)

df_volume_imp = (
    df_volume_imp
    .join(
        df_produtos_top10_imp,
        df_volume_imp.co_ncm == df_produtos_top10_imp.produto_ncm,
        how="inner"
    )
).drop("produto_ncm")


In [0]:
df_volume_imp.display()

In [0]:
salva_tabela(df_volume_imp,"view_painel3_volume_imp",PATH_GOLD)

## KPIs - Painel 4

### Dimensões

In [0]:
top10_final.createOrReplaceTempView("top10_final")

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW tmp_top10_final AS
SELECT * FROM top10_final;

In [0]:
%sql


-- top10 ncm filtro
CREATE OR REPLACE TEMPORARY VIEW tmp_top10_final AS
SELECT CO_NCM FROM (
  VALUES 
    ('02023000'),
    ('12019000'),
    ('17011400'),
    ('26011100'),
    ('27011200'),
    ('27090010'),
    ('27101921'),
    ('31042090'),
    ('84119100'),
    ('09011110')
) AS t(CO_NCM);

-- top10 ncm filtro

-- bridge
CREATE OR REPLACE TEMPORARY VIEW tmp_bridge AS 
SELECT ncm, cnae
FROM silver.corr_ncm_cnae;

-- dim_cnae
CREATE OR REPLACE TEMPORARY VIEW tmp_dim_cnae AS
WITH cnae_5dig AS (
  SELECT DISTINCT
    codigo_cnae AS codigo_cnae_original,
    SUBSTRING(LPAD(CAST(codigo_cnae AS STRING), 7, '0'), 1, 5) AS codigo_cnae_5dig
  FROM silver.cnaes
  WHERE codigo_cnae IS NOT NULL
)
SELECT 
  c5.codigo_cnae_5dig AS codigo_cnae,
  FIRST(cn.descricao_cnae) AS descricao_cnae,
  SUBSTRING(c5.codigo_cnae_5dig, 1, 2) AS cnae_divisao,
  COUNT(DISTINCT c5.codigo_cnae_original) AS qtd_subclasses
FROM cnae_5dig c5
INNER JOIN silver.cnaes cn 
  ON c5.codigo_cnae_original = cn.codigo_cnae
GROUP BY c5.codigo_cnae_5dig;

-- dim_uf
CREATE OR REPLACE TEMPORARY VIEW tmp_dim_uf AS
SELECT DISTINCT
  sg_uf,
  no_uf,
  no_regiao,
  co_uf
FROM silver.uf;

-- dim_porte
CREATE OR REPLACE TEMPORARY VIEW tmp_dim_porte AS
SELECT * FROM (
  VALUES 
    ('00', 'Não Informado'),
    ('01', 'Microempresa'),
    ('03', 'Empresa de Pequeno Porte'),
    ('05', 'Demais')
) AS t(codigo_porte, descricao_porte);

-------------- dim_cnm
CREATE OR REPLACE TEMPORARY VIEW tmp_dim_ncm AS
SELECT DISTINCT
  co_ncm,
  no_ncm_por,
  no_ncm_ing,
  co_sh6,
  co_fat_agreg
FROM silver.ncm;

-------------- fato_empresa_top10
CREATE OR REPLACE TEMPORARY VIEW tmp_fato_empresas_top10 AS
WITH 
-- CTE 1: CNAEs do universo top10
cnaes_top10 AS (
  SELECT DISTINCT cnae AS cnae_classe_5dig
  FROM tmp_bridge
  WHERE ncm IN (SELECT co_ncm FROM tmp_top10_final) --filtroaqui
),

-- CTE 2: Empresas ativas com CNAE extraído (5 dígitos)
cnpj_cnae5 AS (
  SELECT 
    e.cnpj_basico,
    e.uf,
    e.identificador_matriz_filial,
    e.data_inicio_atividade,
    LPAD(CAST(emp.porte_empresa AS STRING), 2, '0') AS codigo_porte,
    CAST(SUBSTRING(CAST(e.cnae_fiscal_principal AS STRING), 1, 5) AS INT) AS cnae_classe_5dig
  FROM silver.estabelecimentos e
  INNER JOIN silver.empresas emp 
    ON e.cnpj_basico = emp.cnpj_basico
  WHERE 
    e.situacao_cadastral = 2
    AND e.uf IS NOT NULL
    AND e.cnae_fiscal_principal IS NOT NULL
),

-- CTE 3: Volume Exportação por CNAE + UF
vol_exp AS (
  SELECT 
    b.cnae AS cnae_classe_5dig,
    ex.sigla_uf_ncm AS uf,
    SUM(ex.vl_fob) AS vl_fob_exp,
    SUM(ex.kg_liquido) AS kg_liquido_exp,
    COUNT(*) AS qtd_operacoes_exp
  FROM tmp_bridge b
  INNER JOIN tmp_top10_final t ON b.ncm = t.co_ncm
  INNER JOIN silver.exp ex ON b.ncm = ex.co_ncm
  GROUP BY b.cnae, ex.sigla_uf_ncm
),

-- CTE 4: Volume Importação por CNAE + UF
vol_imp AS (
  SELECT 
    b.cnae AS cnae_classe_5dig,
    im.sigla_uf_ncm AS uf,
    SUM(im.vl_fob) AS vl_fob_imp,
    SUM(im.kg_liquido) AS kg_liquido_imp,
    COUNT(*) AS qtd_operacoes_imp
  FROM tmp_bridge b
  INNER JOIN tmp_top10_final t ON b.ncm = t.co_ncm
  INNER JOIN silver.imp im ON b.ncm = im.co_ncm
  GROUP BY b.cnae, im.sigla_uf_ncm
)

-- QUERY FINAL
SELECT 
  c.cnae_classe_5dig AS codigo_cnae,
  c.uf,
  c.codigo_porte,
  
  COUNT(DISTINCT c.cnpj_basico) AS qtd_empresas,
  COUNT(DISTINCT CASE WHEN c.identificador_matriz_filial = 1 THEN c.cnpj_basico END) AS qtd_matrizes,
  
  ROUND(AVG(DATEDIFF(CURRENT_DATE(), c.data_inicio_atividade) / 365.25), 2) AS longevidade_media_anos,
  ROUND(MIN(DATEDIFF(CURRENT_DATE(), c.data_inicio_atividade) / 365.25), 2) AS longevidade_min_anos,
  ROUND(MAX(DATEDIFF(CURRENT_DATE(), c.data_inicio_atividade) / 365.25), 2) AS longevidade_max_anos,
  
  COALESCE(MAX(exp.vl_fob_exp), 0) AS volume_exportacao,
  COALESCE(MAX(exp.kg_liquido_exp), 0) AS kg_exportacao,
  COALESCE(MAX(exp.qtd_operacoes_exp), 0) AS operacoes_exportacao,
  COALESCE(MAX(imp.vl_fob_imp), 0) AS volume_importacao,
  COALESCE(MAX(imp.kg_liquido_imp), 0) AS kg_importacao,
  COALESCE(MAX(imp.qtd_operacoes_imp), 0) AS operacoes_importacao,
  
  CURRENT_TIMESTAMP() AS _gold_insert_ts

FROM cnpj_cnae5 c
INNER JOIN cnaes_top10 ct 
  ON c.cnae_classe_5dig = ct.cnae_classe_5dig
LEFT JOIN vol_exp exp 
  ON c.cnae_classe_5dig = exp.cnae_classe_5dig AND c.uf = exp.uf
LEFT JOIN vol_imp imp 
  ON c.cnae_classe_5dig = imp.cnae_classe_5dig AND c.uf = imp.uf
GROUP BY 
  c.cnae_classe_5dig,
  c.uf,
  c.codigo_porte;

### Views pbi

In [0]:
%sql
-- TABELA PBI 1: Dimensão NCM - top10 - aux

DROP TABLE IF EXISTS gold.painel4_dim_ncm_top10;
DROP TABLE IF EXISTS gold.view_painel4_dim_ncm_top10;

CREATE TABLE gold.view_painel4_dim_ncm_top10
USING DELTA
LOCATION 'abfss://gold@aceleragrupo5sa.dfs.core.windows.net/view_painel4_dim_ncm_top10'
AS
SELECT 
  t.co_ncm,
  n.no_ncm_por AS descricao_ncm
FROM tmp_top10_final t
LEFT JOIN tmp_dim_ncm n 
  ON t.co_ncm = n.co_ncm
ORDER BY t.co_ncm;

In [0]:
%sql
-- TABELA PBI 2: Bridge filtrada (NCM ↔ CNAE) - aux
DROP TABLE IF EXISTS gold.view_painel4_bridge_top10;

CREATE TABLE gold.view_painel4_bridge_top10
USING DELTA
LOCATION 'abfss://gold@aceleragrupo5sa.dfs.core.windows.net/view_painel4_bridge_top10'
AS
SELECT DISTINCT 
  CAST(b.ncm AS STRING) AS co_ncm,
  b.cnae AS codigo_cnae
FROM tmp_bridge b
INNER JOIN tmp_top10_final t 
  ON b.ncm = t.co_ncm;

In [0]:
%sql
-- TABELA PBI 3: Empresas por UF — Visual 1 (Mapa)

DROP TABLE IF EXISTS gold.painel4_empresas_uf;
DROP TABLE IF EXISTS gold.view_painel4_empresas_uf;

CREATE TABLE gold.view_painel4_empresas_uf
USING DELTA
LOCATION 'abfss://gold@aceleragrupo5sa.dfs.core.windows.net/view_painel4_empresas_uf'
AS
SELECT 
  CAST(f.codigo_cnae AS STRING) AS codigo_cnae,
  f.uf,
  u.no_uf,
  u.no_regiao,
  SUM(f.qtd_empresas) AS total_empresas,
  SUM(f.qtd_matrizes) AS total_matrizes,
  ROUND(
    SUM(f.longevidade_media_anos * f.qtd_empresas) / NULLIF(SUM(f.qtd_empresas), 0),
    2
  ) AS longevidade_media
FROM tmp_fato_empresas_top10 f
INNER JOIN tmp_dim_uf u 
  ON f.uf = u.sg_uf
GROUP BY 
  f.codigo_cnae,
  f.uf,
  u.no_uf,
  u.no_regiao
ORDER BY total_empresas DESC;

In [0]:
%sql

-- TABELA PBI 4: Perfil de Porte — Visual 2 

DROP TABLE IF EXISTS gold.painel4_perfil_porte;
DROP TABLE IF EXISTS gold.view_painel4_perfil_porte;

CREATE TABLE gold.view_painel4_perfil_porte
USING DELTA
LOCATION 'abfss://gold@aceleragrupo5sa.dfs.core.windows.net/view_painel4_perfil_porte'
AS
SELECT 
  CAST(f.codigo_cnae AS STRING) AS codigo_cnae,
  f.codigo_porte,
  p.descricao_porte,
  SUM(f.qtd_empresas) AS total_empresas,
  SUM(f.qtd_matrizes) AS total_matrizes
FROM tmp_fato_empresas_top10 f
INNER JOIN tmp_dim_porte p 
  ON f.codigo_porte = p.codigo_porte
GROUP BY 
  f.codigo_cnae,
  f.codigo_porte,
  p.descricao_porte
ORDER BY 
  f.codigo_cnae,
  f.codigo_porte;

In [0]:
%sql

-- TABELA PBI 5: Top CNAEs por Região — Visual 3 

DROP TABLE IF EXISTS gold.view_painel4_cnaes_por_regiao;

CREATE TABLE gold.view_painel4_cnaes_por_regiao
USING DELTA
LOCATION 'abfss://gold@aceleragrupo5sa.dfs.core.windows.net/view_painel4_cnaes_por_regiao'
AS
WITH ranking_cnaes AS (
  SELECT 
    CAST(f.codigo_cnae AS STRING) AS codigo_cnae,
    c.descricao_cnae AS nome_classe_cnae,
    c.cnae_divisao,
    u.no_regiao,
    SUM(f.qtd_empresas) AS total_empresas,
    SUM(f.qtd_matrizes) AS total_matrizes,
    ROUND(
      SUM(f.longevidade_media_anos * f.qtd_empresas) / NULLIF(SUM(f.qtd_empresas), 0),
      2
    ) AS longevidade_media,
    ROW_NUMBER() OVER (
      PARTITION BY u.no_regiao 
      ORDER BY SUM(f.qtd_empresas) DESC
    ) AS ranking
  FROM tmp_fato_empresas_top10 f
  INNER JOIN tmp_dim_uf u 
    ON f.uf = u.sg_uf
  INNER JOIN tmp_dim_cnae c 
    ON CAST(f.codigo_cnae AS STRING) = c.codigo_cnae
  WHERE u.no_regiao != 'REGIÃO NÃO DECLARADA'
    AND u.no_regiao IS NOT NULL
  GROUP BY 
    f.codigo_cnae,
    c.descricao_cnae,
    c.cnae_divisao,
    u.no_regiao
)
SELECT 
  codigo_cnae,
  nome_classe_cnae,
  cnae_divisao,
  no_regiao,
  total_empresas,
  total_matrizes,
  longevidade_media,
  ranking
FROM ranking_cnaes
WHERE ranking <= 10
ORDER BY no_regiao, ranking;

In [0]:
%sql

-- TABELA PBI 6: Longevidade por CNAE — Vis 4 (Tabela)

DROP TABLE IF EXISTS gold.view_painel4_longevidade_cnae;
DROP TABLE IF EXISTS gold.view_painel4_longevidade_cnae;


CREATE TABLE gold.view_painel4_longevidade_cnae
USING DELTA
LOCATION 'abfss://gold@aceleragrupo5sa.dfs.core.windows.net/view_painel4_longevidade_cnae'
AS
SELECT 
  CAST(f.codigo_cnae AS STRING) AS codigo_cnae,
  c.descricao_cnae AS nome_classe_cnae,
  c.cnae_divisao,
  ROUND(
    SUM(f.longevidade_media_anos * f.qtd_empresas) / NULLIF(SUM(f.qtd_empresas), 0),
    2
  ) AS longevidade_media_anos,
  SUM(f.qtd_empresas) AS total_empresas,
  SUM(f.qtd_matrizes) AS total_matrizes,
  CASE 
    WHEN SUM(f.longevidade_media_anos * f.qtd_empresas) / NULLIF(SUM(f.qtd_empresas), 0) >= 20 
      THEN 'Setor Maduro (20+ anos)'
    WHEN SUM(f.longevidade_media_anos * f.qtd_empresas) / NULLIF(SUM(f.qtd_empresas), 0) >= 10 
      THEN 'Setor Estabelecido (10-20 anos)'
    WHEN SUM(f.longevidade_media_anos * f.qtd_empresas) / NULLIF(SUM(f.qtd_empresas), 0) >= 5 
      THEN 'Setor Emergente (5-10 anos)'
    ELSE 'Setor Novo (< 5 anos)'
  END AS classificacao_maturidade,
  SUM(f.volume_exportacao) AS volume_exportacao_total,
  SUM(f.volume_importacao) AS volume_importacao_total
FROM tmp_fato_empresas_top10 f
INNER JOIN tmp_dim_cnae c 
  ON CAST(f.codigo_cnae AS STRING) = c.codigo_cnae
GROUP BY 
  f.codigo_cnae,
  c.descricao_cnae,
  c.cnae_divisao
ORDER BY longevidade_media_anos DESC;

## KPIs - Painel 5

### Dataframes auxiliares

#### Dim empresas+estabelecimentos+municipios

##### df_empresas_full


In [0]:
# df_empresas_full

# se não rodar a carga full do CNPJ, isso pode vir vazio.
try:
    df_estab                      = spark.sql("select * from hive_metastore.silver.estabelecimentos")
    df_empresas                   = spark.sql("select * from hive_metastore.silver.empresas")
    df_municipios                 = spark.sql("select * from hive_metastore.silver.municipios")
    
    df_empresas_clean = df_empresas.withColumn("capital_social_clean", F.regexp_replace("capital_social", ",", ".").cast("double"))
    
    df_empresas_full = df_estab.alias("e") \
    .join(df_empresas_clean.alias("emp"), "cnpj_basico", "left") \
    .join(df_municipios.alias("m"), F.col("e.municipio") == F.col("m.codigo_municipio"), "left") \
    .select(
        F.col("e.cnpj_basico"),
        F.col("e.cnpj_ordem"),
        F.col("e.cnpj_dv"),
        F.concat_ws("", F.col("e.cnpj_basico"), F.col("e.cnpj_ordem"), F.col("e.cnpj_dv")).alias("cnpj_completo"),
        F.col("emp.razao_social"),
        F.col("e.nome_fantasia"),
        F.col("e.cnae_fiscal_principal").cast("int").alias("co_cnae"),
        F.col("e.uf"),
        F.col("m.descricao_municipio").alias("municipio"),
        F.col("emp.porte_empresa"),
        F.col("emp.capital_social_clean").alias("capital_social"),
        F.col("e.situacao_cadastral").cast("int").alias("situacao_cadastral"),
        F.col("e.data_inicio_atividade")
    )

    df_empresas_full.printSchema()  
except Exception as e:
    print(f"Erro: {e}")

#### Dim NCM Full

##### dim_ncm_full

In [0]:
# -------------------------------------------------------------------------
# 3. dim ncm enriched 

df_ncm               = spark.sql("select * from hive_metastore.silver.ncm")
df_isic              = spark.sql("select * from hive_metastore.silver.ncm_isic")
df_cuci              = spark.sql("select * from hive_metastore.silver.ncm_cuci")

# alias para facilitar join
df_ncm = df_ncm.alias("n")
df_isic = df_isic.alias("i")
df_cuci = df_cuci.alias("c")

# assumindo schema da bronze
df_ncm_full = df_ncm \
    .join(df_isic, F.col("n.co_isic_classe") == F.col("i.co_isic_classe"), "left") \
    .join(df_cuci, F.col("n.co_cuci_item") == F.col("c.co_cuci_item"), "left") 

# df_ncm_silver = df_ncm_full.select(
#     F.col("n.co_ncm").cast("int").alias("co_ncm"),
#     F.col("n.NO_NCM_POR").alias("desc_ncm"),
#     F.col("n.CO_ISIC_CLASSE").alias("co_isic_classe"),
#     F.col("i.NO_ISIC_CLASSE").alias("desc_isic_classe"),
#     F.col("i.NO_ISIC_SECAO").alias("desc_isic_secao"),
#     F.col("c.NO_CUCI_GRUPO").alias("desc_cuci_grupo")
# )

df_ncm_full = df_ncm_full.select(
    F.col("n.co_ncm"),
    F.col("n.no_ncm_por"),
    F.col("n.co_isic_classe"),
    F.col("i.no_isic_classe"),
    F.col("i.no_isic_secao"),
    F.col("c.no_cuci_grupo")
)

In [0]:
df_ncm_full.printSchema()

### Processamento dos Dataframes principais

#### 1. Top10 com valores

In [0]:
# ==============================================================================
# 1. RECUPERAÇÃO DE VALORES - O top10_final só tem o código, precisamos do vl_fob
df_valores_top10 = df_exp.join(
    top10_final,
    F.lpad(df_exp.co_ncm, 8, '0') == F.lpad(top10_final.CO_NCM, 8, '0'),
    "inner"
).groupBy(df_exp.co_ncm) \
 .agg(F.sum("vl_fob").alias("vl_fob_recuperado"))

In [0]:
display(df_valores_top10)

In [0]:
df_valores_top10.printSchema()

#### 2. Tabela Intermediária - Bridge

In [0]:
df_bridge_silver = spark.sql("select * from hive_metastore.silver.corr_ncm_cnae")

In [0]:
# ==============================================================================
# 2. CRUZAMENTO: NCM -> CNAE ==== Bridge
df_ncm_cnae = df_valores_top10.join(
    df_bridge_silver,
    F.lpad(df_valores_top10.co_ncm, 8, '0') == F.lpad(df_bridge_silver.ncm.cast("string"), 8, '0'),
    "inner"
)

#### 3. Cruzamento: CNAE -> Empresas 

In [0]:
# ==============================================================================
# 3. CRUZAMENTO: CNAE -> EMPRESAS (Inteligência de Mercado)
# filtramos apenas situacao_cadastral = 2, ou seja, Ativas
df_oportunidade = df_ncm_cnae.join(
    df_empresas_full.filter(F.col("situacao_cadastral") == 2),
    F.lpad(df_ncm_cnae.cnae.cast("string"), 5, "0") == F.lpad(df_empresas_full.co_cnae.cast("string"), 5, "0"),
    "inner"
)

In [0]:
display(df_oportunidade)

#### 4. Agregação Final

In [0]:
# ==============================================================================
# 4. AGREGAÇÃO FINAL (KPIs)
df_painel_5_gold = df_oportunidade.groupBy("co_ncm", "vl_fob_recuperado") \
    .agg(
        F.countDistinct("cnpj_basico").alias("qtd_empresas_habilitadas"),
        F.round(
            F.avg(
                F.datediff(F.current_date(), F.col("data_inicio_atividade")) / 365
            ), 1
        ).alias("longevidade_media_anos")
    ) \
    .join(df_ncm_full, "co_ncm", "left") \
    .select(
        F.col("no_ncm_por").alias("desc_ncm"),
        F.col("vl_fob_recuperado").alias("total_exportado"),
        F.col("qtd_empresas_habilitadas"),
        F.col("longevidade_media_anos")
    ) \
    .orderBy(F.col("total_exportado").desc())

In [0]:
display(df_painel_5_gold)

### Persistencia na Hive

In [0]:
salva_tabela(df_painel_5_gold, "view_painel5_oportunidades", PATH_GOLD)

In [0]:
(
    df_painel_5_gold
    .write
    .mode("overwrite")
    .format("delta")
    .option("overwriteSchema", "true")
    .saveAsTable("hive_metastore.gold.view_painel5_oportunidades")
)

### Data Quality da Bridge

In [0]:
# ============================================================================
# AUDITORIA DE QUALIDADE DA BRIDGE (NCM x CNAE)

# 1. Preparando os dados -garantindo que não haja erro de tipagem/zeros
# Usamos a base de exportação (df_exp ou df_exp_clean) como referência
df_audit_exp = spark.table("hive_metastore.silver.exp")\
    .withColumn("ncm_key", F.lpad(F.col("co_ncm").cast("string"), 8, "0"))

df_audit_bridge = df_bridge_silver \
    .withColumn("ncm_key", F.lpad(F.col("NCM").cast("string"), 8, "0"))

# 2. Calculando totais Gerais
total_vl_fob = df_audit_exp.agg(F.sum("vl_fob")).collect()[0][0]
total_qtd_ncm = df_audit_exp.select("ncm_key").distinct().count()

# 3. Fazendo o match sem duplicar linhas
# Pegamos apenas a lista de NCMs únicos que a Bridge conhece
df_bridge_unique_ncms = df_audit_bridge.select("ncm_key").distinct()

# Fazemos o join da exportação com essa lista única 
# Assim, se um NCM tem 10 CNAEs, ele continua valendo apenas 1 linha de dinheiro aqui.
df_match_clean = df_audit_exp.join(df_bridge_unique_ncms, "ncm_key", "inner")

# 4. Calculando totais mapeados 
mapped_vl_fob = df_match_clean.agg(F.sum("vl_fob")).collect()[0][0]
mapped_qtd_ncm = df_match_clean.select("ncm_key").distinct().count()

# 5. Calculando fator de complexidade (Aqui usamos a bridge completa para ver as duplicações)
complexity_factor = df_audit_bridge.groupBy("ncm_key") \
    .count() \
    .agg(F.avg("count")) \
    .collect()[0][0]

In [0]:
# ============================================================================
# EXIBIÇÃO DOS RESULTADOS

print(f"--- RELATÓRIO DE COBERTURA DA TABELA BRIDGE (CORRIGIDO) ---")
print(f"1. Cobertura de Catálogo (Códigos NCM): {(mapped_qtd_ncm / total_qtd_ncm * 100):.2f}%")
print(f"   (De {total_qtd_ncm} produtos distintos, identificamos {mapped_qtd_ncm})")
print(f"")
print(f"2. Complexidade da Modelagem (NCM -> CNAEs): {complexity_factor:.2f}x")
print(f"   (Em média, cada Produto pode ser fabricado por {complexity_factor:.2f} tipos de indústrias diferentes.")

# DataFrame para Display
data = [
    ("Cobertura de Produtos (NCMs)", f"{(mapped_qtd_ncm / total_qtd_ncm * 100):.2f}%", "Alta"),
    ("Média de CNAEs por Produto", f"{complexity_factor:.2f}", "Complexidade N:N")
]
spark.createDataFrame(data, ["Metrica", "Valor", "Status"]).display()

In [0]:
%sql
-- ═══════════════════════════════════════════════════════════════════
-- VALIDAÇÃO 2: COBERTURA DE VALOR FOB
-- Critério de Sucesso: >60% do valor FOB com empresas mapeadas

SELECT 
    ROUND(SUM(CASE WHEN b.NCM IS NOT NULL THEN e.vl_fob ELSE 0 END) * 100.0 / SUM(e.vl_fob), 2) AS perc_fob_com_match_ncm,
    ROUND(SUM(CASE WHEN b.NCM IS NULL THEN e.vl_fob ELSE 0 END) * 100.0 / SUM(e.vl_fob), 2) AS perc_fob_sem_match_ncm,
    SUM(CASE WHEN b.NCM IS NOT NULL THEN e.vl_fob ELSE 0 END) / 1000000000 AS vl_fob_com_match_bilhoes,
    SUM(CASE WHEN b.NCM IS NULL THEN e.vl_fob ELSE 0 END) / 1000000000 AS vl_fob_sem_match_bilhoes
FROM 
    silver.exp e
LEFT JOIN 
    silver.corr_ncm_cnae b ON e.co_ncm = b.NCM
WHERE 
    e.ano >= 2023;